In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')
#data PWT 10.01 in https://www.rug.nl/ggdc/productivity/pwt/ 1950-2019

#project2 conuntry ”France”, ”Germany”, ”Canada”, ”Italy”, ”Japan”, ”United Kingdom”, ”United States”, 1995-2019

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany","Canada","Italy","Japan",
                                             "United Kingdam", "United States"])][
                                                 ['year', 'country', 'rgdpna', 
                                                  'rkna','pop', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()


# Calculate additional columns
data['alpha'] = 1 - data['labsh'] # Capital share
data['y_n'] = data['rgdpna'] / data['pop'] # GDP per worker Y/N
data['y_n'] = np.log(data['y_n'])  # ln(Y/N)
data['hours'] = data['emp'] * data['avh']  # L
data['lab_term'] = data['hours'] / data['pop']  # L/N
data['lab_term'] = np. log(data['lab_term']) # ln(L/N)
# ln(TFP) = ln(Y/N) - alpha * ln(L/N)
data['tfp_term'] = data['y_n'] - data['alpha'] * data['lab_term'] #tfp
data['cap_term'] = data['alpha'] * data['lab_term']  # cap


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('country')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_n'].diff() * 100)  # Growth rate 
data['tfp'] = (grouped_data['tfp_term'].diff() * 100) #TFP Growth
data['cap'] = (grouped_data['cap_term'].diff() * 100) #Capital Deepning

# Remove missing values
data = data.dropna()

# Calculate summary statistics #
summary = data.groupby('country').agg({'g': 'mean',
                                       'alpha': "mean",
                                       'tfp':'mean',
                                       'cap':'mean'})


# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['alpha']
summary['TFP Share'] = summary['tfp']
summary['Capital Deepning']=summary['cap']
# Print output
print(summary)






                      g     alpha       tfp  Growth Rate  Capital Share  \
country                                                                   
Canada         0.887700  0.346190  0.222313     0.887700       0.346190   
France         0.829236  0.381194  0.147461     0.829236       0.381194   
Germany        0.661055  0.370730  0.497941     0.661055       0.370730   
Italy         -0.037181  0.484835 -1.728427    -0.037181       0.484835   
Japan          0.710956  0.430601  0.232745     0.710956       0.430601   
United States  1.536985  0.393260  1.005214     1.536985       0.393260   

               TFP Share  
country                   
Canada          0.222313  
France          0.147461  
Germany         0.497941  
Italy          -1.728427  
Japan           0.232745  
United States   1.005214  
